In [1]:
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [2]:
data = pd.read_csv("data.csv")

In [3]:
MAP = {
    "r1":["r2","r3"],
    "r2":["r1","r4"],
    "r3":["r1","r7"],
    "r4":["r2","r8"],
    "r5":["r6","r9","c3"],
    "r6":["r5","c3"],
    "r7":["r3","c1"],
    "r8":["r4","r9"],
    "r9":["r5","r8","r13"],
    "r10":["c3"],
    "r11":["c3"],
    "r12":["outside","r22"],
    "r13":["r9","r24"],
    "r14":["r24"],
    "r15":["c3"],
    "r16":["c3"],
    "r17":["c3"],
    "r18":["c3"],
    "r19":["c3"],
    "r20":["c3"],
    "r21":["c3"],
    "r22":["r12","r25"],
    "r23":["r24"],
    "r24":["r13","r14","r23"],
    "r25":["r22","r26","c1"],
    "r26":["r25","r27"],
    "r27":["r26","r32"],
    "r28":["c4"],
    "r29":["c4","r30"],
    "r30":["r29"],
    "r31":["r32"],
    "r32":["r27","r31","r33"],
    "r33":["r32"],
    "r34":["c2"],
    "r35":["c4"],
    "c1":["r7","r25","c2"],
    "c2":["c1","r34","c4"],
    "c3": ["r5","r6","r10","r11","r15","r16","r17","r18","r19","r20","r21","o1"],
    "c4":["r29","c2","r35","r28","o1"],
    "o1":["c3","c4"],
    "outside":["r12"]  
}
censor_DF =  {
    "r1": "us3", #us for unreliable_sensor
    "r5": "rs2" ,#rs for reliable_sensor
    "r8": "ds1", #ds for door sensor
    "r9": "ds1",
    "r16": "rs1",
    "r24" : "us4",
    "r25" : "rs3",
    "r26" : "ds3", 
    "r27" : "ds3",
    "r31" : "rs4",
    "r35" : "ds4",
    "c1" : "ds2",
    "c2" : "ds2",
    "c3" : "us2",
    "c4" : "ds4",
    "o1" : "us1"
}

In [4]:
index1 = data[data.time == "08:01:00"].index.tolist()[0] #  get in entrance ends
index2 = data[data.time == "08:05:00"].index.tolist()[0] # People tends to be stable
index3 = data[data.time=="17:30:00"].index.tolist()[0] # leave work starts
index4 = data[data.time=="17:41:00"].index.tolist()[0] # leave work ends

In [5]:
data1 = data.iloc[0:index1] # 08:00 - 08:01
data2 = data.iloc[index1:index2].reset_index(drop=True) #08:01 - 08:05
data3 = data.iloc[index2:index3].reset_index(drop=True) # 08:05 - 17:30
data4 = data.iloc[index3:index4].reset_index(drop=True) # 17:30 - 17:41
data5 = data.iloc[index4:].reset_index(drop=True)
data_lst = [data1,data2,data3,data4,data5]

In [6]:
# self_trans means the probability of a person keeps staying in the same area
self_trans = {key:[] for key in MAP.keys()} 

for sub_data in data_lst:
    for place in MAP.keys():
        col_data = sub_data[place] 
        stay = 0
        go = 0
        for i in range(1,col_data.shape[0]):
            if(col_data[i]>=col_data[i-1]):
                stay += col_data[i-1]
            else:
                stay += col_data[i]
                go += col_data[i-1] - col_data[i]
        total = stay+go
        if(total!=0):
            self_trans[place].append(stay/total)
        else:
            self_trans[place].append(0)
                    
            
print(self_trans)

{'r1': [0, 1.0, 0.9260340632603407, 0.3333333333333333, 0], 'r2': [0, 0, 0.6417910447761194, 0.0, 0], 'r3': [0, 0.9, 0.9156235512285582, 0.3333333333333333, 0], 'r4': [0, 0, 0.8225602027883396, 0.5, 0], 'r5': [0, 0, 0.6028169014084507, 0.0, 0], 'r6': [0, 0, 0.9585492227979274, 0, 0], 'r7': [0, 0.0, 0.9445903361344538, 0.5, 0], 'r8': [0, 0, 0.2654320987654321, 0, 0], 'r9': [0, 0, 0.8112058465286236, 0, 0], 'r10': [0, 1.0, 0.985493898227032, 0, 0], 'r11': [0, 1.0, 0.9837587006960556, 0.5, 0], 'r12': [0.8888888888888888, 0, 0.9613259668508287, 0.6, 0], 'r13': [0, 0, 0.8971848225214198, 0, 0], 'r14': [0, 0, 0.9796251018744906, 0, 0], 'r15': [0, 1.0, 0.9862700228832952, 0, 0], 'r16': [0, 0.7142857142857143, 0.9833518312985572, 0.9354838709677419, 0], 'r17': [0, 0.6666666666666666, 0.972939729397294, 0, 0], 'r18': [0, 1.0, 0.9826539462272333, 0, 0], 'r19': [0, 0.5, 0.979154603358425, 0.5, 0], 'r20': [0, 0, 0.9761904761904762, 0, 0], 'r21': [0, 0, 0.9595687331536388, 0, 0], 'r22': [1.0, 0.166

## Find Neighbour Transition

In [7]:
def n_step_neighbour(node,n,G):
    neighbour_list = []
    neighbour_list = neighbour_list + G[node]
    while n > 1:
        for new_node in neighbour_list:
            neighbour_list = neighbour_list+ G[new_node]
        n = n -1 
    n_list = list(set(neighbour_list))
    n_list.remove(node)
    return(n_list)

In [8]:
neighbour_MAP = {key:n_step_neighbour(key,2,MAP) for key in MAP.keys()}

In [9]:
neighbour_MAP

{'r1': ['r4', 'r2', 'r3', 'r7'],
 'r2': ['r8', 'r1', 'r4', 'r3'],
 'r3': ['c1', 'r1', 'r2', 'r7'],
 'r4': ['r9', 'r8', 'r1', 'r2'],
 'r5': ['r13',
  'r21',
  'o1',
  'r18',
  'r11',
  'r9',
  'r6',
  'r15',
  'r8',
  'r10',
  'c3',
  'r16',
  'r19',
  'r17',
  'r20'],
 'r6': ['r21',
  'o1',
  'r18',
  'r11',
  'r9',
  'r15',
  'r10',
  'r16',
  'c3',
  'r19',
  'r17',
  'r20',
  'r5'],
 'r7': ['c2', 'c1', 'r1', 'r25', 'r3'],
 'r8': ['r13', 'r9', 'r2', 'r4', 'r5'],
 'r9': ['r13', 'r24', 'r6', 'r8', 'c3', 'r4', 'r5'],
 'r10': ['r21',
  'o1',
  'r18',
  'r11',
  'r6',
  'r15',
  'r16',
  'c3',
  'r19',
  'r17',
  'r20',
  'r5'],
 'r11': ['r21',
  'o1',
  'r18',
  'r6',
  'r15',
  'r10',
  'r16',
  'c3',
  'r19',
  'r17',
  'r20',
  'r5'],
 'r12': ['r22', 'outside', 'r25'],
 'r13': ['r24', 'r23', 'r14', 'r9', 'r8', 'r5'],
 'r14': ['r23', 'r13', 'r24'],
 'r15': ['r21',
  'o1',
  'r18',
  'r11',
  'r6',
  'r10',
  'r16',
  'c3',
  'r19',
  'r17',
  'r20',
  'r5'],
 'r16': ['r21',
  'o1',
  '

In [10]:
# neigh_trans means the probability of a person transfers to the neighbourhood
neigh_trans = {key:{} for key in MAP.keys()} 
for key,neighbours in neighbour_MAP.items():
    for neighbour in neighbours:
        neigh_trans[key][neighbour]=[]

In [11]:
neigh_trans

{'r1': {'r4': [], 'r2': [], 'r3': [], 'r7': []},
 'r2': {'r8': [], 'r1': [], 'r4': [], 'r3': []},
 'r3': {'c1': [], 'r1': [], 'r2': [], 'r7': []},
 'r4': {'r9': [], 'r8': [], 'r1': [], 'r2': []},
 'r5': {'r13': [],
  'r21': [],
  'o1': [],
  'r18': [],
  'r11': [],
  'r9': [],
  'r6': [],
  'r15': [],
  'r8': [],
  'r10': [],
  'c3': [],
  'r16': [],
  'r19': [],
  'r17': [],
  'r20': []},
 'r6': {'r21': [],
  'o1': [],
  'r18': [],
  'r11': [],
  'r9': [],
  'r15': [],
  'r10': [],
  'r16': [],
  'c3': [],
  'r19': [],
  'r17': [],
  'r20': [],
  'r5': []},
 'r7': {'c2': [], 'c1': [], 'r1': [], 'r25': [], 'r3': []},
 'r8': {'r13': [], 'r9': [], 'r2': [], 'r4': [], 'r5': []},
 'r9': {'r13': [],
  'r24': [],
  'r6': [],
  'r8': [],
  'c3': [],
  'r4': [],
  'r5': []},
 'r10': {'r21': [],
  'o1': [],
  'r18': [],
  'r11': [],
  'r6': [],
  'r15': [],
  'r16': [],
  'c3': [],
  'r19': [],
  'r17': [],
  'r20': [],
  'r5': []},
 'r11': {'r21': [],
  'o1': [],
  'r18': [],
  'r6': [],
  'r1

In [12]:
for sub_data in data_lst:
    place_data = sub_data.iloc[:,17:58]
    pl_diff = place_data.diff()
    sub_neigh_trans = {key:{} for key in MAP.keys()}
    for key,neighbours in neighbour_MAP.items():
        for neighbour in neighbours:
            sub_neigh_trans[key][neighbour]= 0 
    for place in MAP.keys():
        for i in range(sub_data.shape[0]):
            if(sub_data.iloc[i][place]<sub_data.iloc[i-1][place]): # someone leaves the place
                    for neighbour in neighbour_MAP[place]:
                        if pl_diff.iloc[i][neighbour]>0:
                            sub_neigh_trans[place][neighbour]+=pl_diff.iloc[i][neighbour]
                        else:
                            sub_neigh_trans[place][neighbour]+=0
        neigh_total = sum(sub_neigh_trans[place].values())
        if (neigh_total!=0):
             for neighbour in neighbour_MAP[place]:
                sub_neigh_trans[place][neighbour] = sub_neigh_trans[place][neighbour]/neigh_total  
                neigh_trans[place][neighbour].append(sub_neigh_trans[place][neighbour])
        else:
            for neighbour in neighbour_MAP[place]:
                neigh_trans[place][neighbour].append(0) 

print(neigh_trans)

{'r1': {'r4': [0, 0, 0.10919540229885058, 0.0, 0], 'r2': [0, 0, 0.3160919540229885, 0.5, 0], 'r3': [0, 0, 0.41954022988505746, 0.5, 0], 'r7': [0, 0, 0.15517241379310345, 0.0, 0]}, 'r2': {'r8': [0, 0, 0.07432432432432433, 0.0, 0], 'r1': [0, 0, 0.4527027027027027, 1.0, 0], 'r4': [0, 0, 0.42567567567567566, 0.0, 0], 'r3': [0, 0, 0.0472972972972973, 0.0, 0]}, 'r3': {'c1': [0, 0.0, 0.13973799126637554, 0.3333333333333333, 0], 'r1': [0, 1.0, 0.3537117903930131, 0.3333333333333333, 0], 'r2': [0, 0.0, 0.056768558951965066, 0.0, 0], 'r7': [0, 0.0, 0.4497816593886463, 0.3333333333333333, 0]}, 'r4': {'r9': [0, 0, 0.11949685534591195, 0.0, 0], 'r8': [0, 0, 0.3710691823899371, 0.0, 0], 'r1': [0, 0, 0.08176100628930817, 0.0, 0], 'r2': [0, 0, 0.4276729559748428, 1.0, 0]}, 'r5': {'r13': [0, 0, 0.04040404040404041, 0.0, 0], 'r21': [0, 0, 0.0, 0.0, 0], 'o1': [0, 0, 0.08585858585858586, 0.0, 0], 'r18': [0, 0, 0.0, 0.0, 0], 'r11': [0, 0, 0.020202020202020204, 0.0, 0], 'r9': [0, 0, 0.3181818181818182, 0.0,

In [13]:
true_neigh_trans = copy.deepcopy(neigh_trans)

In [14]:
for place, neighbours in true_neigh_trans.items():
    place_tran_prob = [1-val for val in self_trans[place]]
    for k,v in neighbours.items():
        for i in range(len(v)):
            true_neigh_trans[place][k][i] = true_neigh_trans[place][k][i] * place_tran_prob[i] 
            
            
print(true_neigh_trans)

{'r1': {'r4': [0, 0.0, 0.008076740218698435, 0.0, 0], 'r2': [0, 0.0, 0.023380037475179676, 0.33333333333333337, 0], 'r3': [0, 0.0, 0.0310316861034203, 0.33333333333333337, 0], 'r7': [0, 0.0, 0.011477472942360933, 0.0, 0]}, 'r2': {'r8': [0, 0, 0.026623638563937076, 0.0, 0], 'r1': [0, 0, 0.16216216216216217, 1.0, 0], 'r4': [0, 0, 0.15248083904800325, 0.0, 0], 'r3': [0, 0, 0.016942315449778138, 0.0, 0]}, 'r3': {'c1': [0, 0.0, 0.011790595461511516, 0.22222222222222224, 0], 'r1': [0, 0.09999999999999998, 0.029844944761951025, 0.22222222222222224, 0], 'r2': [0, 0.0, 0.004789929406239054, 0.0, 0], 'r7': [0, 0.0, 0.03795097914174019, 0.22222222222222224, 0]}, 'r4': {'r9': [0, 0, 0.02120349778000973, 0.0, 0], 'r8': [0, 0, 0.06584244047476705, 0.0, 0], 'r1': [0, 0, 0.01450765637579613, 0.0, 0], 'r2': [0, 0, 0.07588620258108746, 0.5, 0]}, 'r5': {'r13': [0, 0, 0.016047801963294923, 0.0, 0], 'r21': [0, 0, 0.0, 0.0, 0], 'o1': [0, 0, 0.03410157917200171, 0.0, 0], 'r18': [0, 0, 0.0, 0.0, 0], 'r11': [0

## Transition Matrix Generation

In [15]:
for i in range(len(data_lst)):
    tran_lst = [] 
    for place in MAP.keys():
        tran_row = {}
        tran_row['place'] = place 
        for col_place in MAP.keys():
            if(place==col_place):
                tran_row[col_place] = round(self_trans[col_place][i],4)
            else:
                if (col_place in true_neigh_trans[place].keys()):
                    tran_row[col_place] = round(true_neigh_trans[place][col_place][i],2)
                else:
                    tran_row[col_place] = 0 
        tran_lst.append(tran_row)
    tran_matrix = pd.DataFrame(tran_lst)
    file_name = "tran_matrix" + str(i) + ".csv"
    tran_matrix.to_csv(file_name)